In [1]:
import os
import json
import numpy as np
import pandas as pd

from silx.io.dictdump import h5todict, dicttoh5

In [2]:
from scipy import constants
from scipy.constants import physical_constants as pc
UNA = pc["Avogadro constant"][0]
UKB = pc["Boltzmann constant"][0]
eV_to_J = constants.eV

In [3]:
cm3_to_m3 = 1e-6

Inputs

In [4]:
T = 1e4
nb_states = 45
# Databases
kin_file = {
  "RVS": "/home/zanardi/Workspace/AirDatabase/HDF5/QCT/Orig/O3_UMN.hdf5",
  "VS": "/home/zanardi/Workspace/AirDatabase/HDF5/QCT/Orig/O4_UMN_VSM.hdf5"
}
mol_file = "./../species/molecule.json"
lev_file = "/home/zanardi/Codes/TC/HyperNet/database/levels/O3_UMN/O2.csv"

In [5]:
# Save path
out_path = "./../"
os.makedirs(out_path, exist_ok=True)

Kinetics database

In [6]:
kin_dtb = {}
kinetics = {k: h5todict(filename) for (k, filename) in kin_file.items()}

> Molecule - Atom collisions

In [7]:
def get_mapping(vqn):
  lev_to_bin = np.zeros_like(vqn)
  vmin, vmax = int(np.amin(vqn)), int(np.amax(vqn))
  for (i, vi) in enumerate(range(vmin, vmax+1)):
    lev_to_bin[vqn == vi] = i
  lev_to_bin -= np.amin(lev_to_bin)
  return lev_to_bin.astype(np.int32)

def construct_prob_mat(vqn):
  mapping = get_mapping(vqn)
  mapping -= np.amin(mapping)
  nb_lev = len(mapping)
  nb_comp = np.amax(mapping) + 1
  indices = (np.arange(nb_lev), mapping)
  P = np.zeros((nb_lev, nb_comp), dtype=int)
  P[indices] = 1
  return P

def q_int(g, e, T):
  return g * np.exp(-e/(UKB*T))

In [8]:
levels = pd.read_csv(lev_file)
vqn, g, e = [levels[k].to_numpy() for k in ("vqn", "g", "E")]
e = (e-np.amin(e))*eV_to_J

In [9]:
P = construct_prob_mat(vqn)
Pq = P * q_int(g, e, T).reshape(-1,1)
Pq /= np.sum(Pq, axis=0, keepdims=True)

In [10]:
ma_dtb = kinetics["RVS"][f"T_{int(T)}_{int(T)}"]["Rates"]

In [11]:
kin_dtb["m-a"] = {}
for (k, rates) in ma_dtb.items():
  if (k == "Diss_Corr"):
    p = "d"
    shape = (nb_states,1,1,1,1)
    # Project
    rates = Pq.T @ rates
  elif (k == "Excit"):
    p = "e"
    shape = (nb_states,1,nb_states,1)
    # Project
    triu_i = np.triu_indices(rates.shape[0])
    rates[triu_i[0],triu_i[1]] = 0.0
    rates = Pq.T @ rates @ P
    # Remove endothermic processes
    triu_i = np.triu_indices(nb_states)
    rates[triu_i[0],triu_i[1]] = 0.0
  else:
    continue
  # Reshape and rescale
  rates = rates.reshape(shape) * cm3_to_m3
  # Store
  kin_dtb["m-a"][p] = rates

> Molecule - Molecule collisions

In [12]:
with open(mol_file, "r") as file:
  molecule = json.load(file)
ei = np.array(molecule["lev"]["e"])
eij = ei.reshape(1,-1) + ei.reshape(-1,1)

In [13]:
mm_dtb = kinetics["VS"][f"T_{int(T)}_{int(T)}"]["Rates"]

In [14]:
kin_dtb["m-m"] = {}
for (k, rates) in mm_dtb.items():
  if (k == "Diss"):
    p = "d"
    shape = (nb_states,nb_states,1,1,1,1)
    rates = rates[...,0]
  elif (k == "DissInel"):
    p = "ed"
    shape = (nb_states,nb_states,1,1,nb_states)
    rates = rates[...,0]
  else:
    p = "e"
    shape = tuple([nb_states]*4)
    # Remove endothermic processes
    for i in range(nb_states):
      for j in range(nb_states):
        rates[i,j][eij[i,j] <= eij] = 0.0
  # Reshape and rescale
  rates = rates.reshape(shape) * cm3_to_m3
  # Store
  if (p in kin_dtb["m-m"]):
    kin_dtb["m-m"][p] += rates
  else:
    kin_dtb["m-m"][p] = rates

> Set Arrhenius parameters

In [15]:
for c in kin_dtb.keys():
  for p in kin_dtb[c].keys():
    zeros = np.zeros_like(kin_dtb[c][p])
    kin_dtb[c][p] = {"A": kin_dtb[c][p], "beta": zeros, "Ta": zeros}

> Save database

In [16]:
dicttoh5(kin_dtb, out_path + "/kinetics.hdf5")